In [0]:
WITH tb_base AS (
  SELECT v.idVendedor,
         ip.idPedido,
         p.dtPedido,
         ap.idAvaliacao,
         ap.vlNota

  FROM silver.olist.vendedor as v
  LEFT JOIN silver.olist.item_pedido as ip ON v.idVendedor = ip.idVendedor
  LEFT JOIN silver.olist.pedido as p ON ip.idPedido = p.idPedido
  LEFT JOIN silver.olist.avaliacao_pedido as ap ON ip.idPedido = ap.idPedido
  WHERE p.dtPedido < '2017-06-01'

),

-- Incluir os atributos da feature store na tabela
tb_feat_avaliacao AS (
  SELECT idVendedor, 
         AVG(vlNota) AS mediaAvaliacao,
         COUNT(vlNota) AS qtdReviewsAteHoje,
         AVG(CASE WHEN  dtv.dtPedido > '2017-06-01' - INTERVAL 28 DAY THEN vlNota END) AS mediaAvaliacao28d,
         COUNT(CASE WHEN  dtv.dtPedido > '2017-06-01' - INTERVAL 28 DAY THEN vlNota END) AS qtdReviewsAteHoje28d,
         (COUNT(CASE WHEN dtv.idAvaliacao IS NULL THEN 1 END) * 1.0 / COUNT(dtv.idPedido)) AS pctPedidosNaoAvaliado
  FROM tb_base AS dtv
  GROUP BY idVendedor
)

SELECT 
      '2017-06-01' AS dtRef,
      *
FROM tb_feat_avaliacao

In [0]:
SELECT date(dtPedido), count(*) FROM silver.olist.pedido
group by ALL
order by 1
